## Data Load

In [1]:
import numpy as np
import pandas as pd
import regex
import re
from PIL import Image
import torch
from torchvision.transforms import v2
from tqdm import tqdm
import os

from collections import Counter
import sys

In [2]:
# config에서 data 폴더 경로 가져오기
data_dir = "/data/ephemeral/home/data"  # 예: "config/data"

# users.csv 경로 만들기
user_data_path = os.path.join(data_dir, "users.csv")
user_df = pd.read_csv(user_data_path)

book_data_path = os.path.join(data_dir, "books.csv")
book_df = pd.read_csv(book_data_path)

train_rating_data_path = os.path.join(data_dir, "train_ratings.csv")
train_rating_df = pd.read_csv(train_rating_data_path)

## User 의 age 피쳐 전처리

In [3]:
# Winsorizing
lower_bound = user_df['age'].quantile(0.05)
upper_bound = user_df['age'].quantile(0.95)

user_df['age_winsor'] = user_df['age'].clip(lower=lower_bound, upper=upper_bound)


In [4]:
user_df['age_winsor'].head()

0     NaN
1    49.0
2     NaN
3    30.0
4    36.0
Name: age_winsor, dtype: float64

In [6]:
bins = [17, 26, 36, 46, 56, 61]  # Winsorized 범위 기준
labels = ['17-25', '26-35', '36-45', '46-55', '56-61']

user_df['age_group'] = pd.cut(user_df['age_winsor'], bins=bins, labels=labels, right=True)


In [13]:
# 연령대별 데이터 개수 확인
age_ratio = user_df['age_group'].value_counts(normalize=True, dropna=False).sort_index()
print(age_ratio)


age_group
17-25    0.129252
26-35    0.173486
36-45    0.113875
46-55    0.081772
56-61    0.055719
NaN      0.445897
Name: proportion, dtype: float64
